In [1]:
import os
import modules.util as util
import math
import numpy as np
import modules.datagenerator as dtgen
import modules.network as network
import modules.loss as loss
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from tensorflow import keras

2023-03-03 20:10:51.904803: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
home=os.path.abspath(os.getcwd())
data_path=os.path.join(home, 'data_testset')

#global data
pairs=[]
classes=[]

for file in os.listdir(data_path):
    classes.append(file)

In [3]:
util.make_pairs(data_path, pairs, classes)

# partition = {'train': np.arange(math.floor(len(pairs)*.6)),
#              'validation': np.arange(math.floor(len(pairs)*.6),math.floor(len(pairs)*.8)),
#              'test': np.arange(math.floor(len(pairs)*.8), math.floor(len(pairs)))}

partition = {'test': np.arange(math.floor(len(pairs)))}

# Generators
test_generator = dtgen.DataGenerator(partition['test'], pairs, batch_size=16)

In [4]:
print(f"Number of Test Pairs: {len(partition['test'])}")

Number of Test Pairs: 1336


In [5]:
siamese_obj = network.SiameseNetwork()
siamese_network = siamese_obj.make_siamese_net()

All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [6]:
siamese_network.compile(loss= loss.loss(1), optimizer=tf.keras.optimizers.Adam(learning_rate=0.5e-3), # 0.5e-6
                    metrics=["accuracy"])

In [7]:
# Re-evaluate the model
loss, acc = siamese_network.evaluate(test_generator, verbose=2)
print("Unrestored model, accuracy: {:5.2f}%".format(100 * acc))


83/83 - 49s - loss: 0.2510 - accuracy: 0.5144 - 49s/epoch - 593ms/step
Unrestored model, accuracy: 51.44%


In [8]:
checkpoint_path = os.path.join(home, 'checkpoints')

# Loads the weights
siamese_network.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = siamese_network.evaluate(test_generator, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

2023-03-03 20:12:34.203595: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open /home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/full_data_3hr/checkpoints: FAILED_PRECONDITION: /home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/full_data_3hr/checkpoints; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


83/83 - 40s - loss: 0.2033 - accuracy: 0.6979 - 40s/epoch - 479ms/step
Restored model, accuracy: 69.79%
